In [9]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt

# Import the Parsers
from openquake.cat.parsers.isf_catalogue_reader import ISFReader
from openquake.cat.parsers.converters import GenericCataloguetoISFParser
from openquake.cat.isc_homogenisor import (HomogenisorPreprocessor,
                                   DynamicHomogenisor,
                                   MagnitudeConversionRule,
                                   DuplicateFinder)


## Load in Catalogue - Limit to ISC, GCMT/HRVD, EHB, NEIC, BJI

In [23]:
isc_parser = ISFReader("inputs/1900-2021-PH_ISF_Catalogue.txt",
                   selected_origin_agencies=["ISC-GEM", "ISC-EHB", "EHB", "ISC", "IDC", "NEIC", "NEIS", "USCGS", "NIED", "GCMT", "GUTE", "PAS"],
                   selected_magnitude_agencies=["ISC-GEM", "ISC-EHB", "EHB", "ISC", "IDC", "NEIC", "NEIS", "USCGS", "NIED", "GCMT", "GUTE", "PAS"])
isc_catalogue = isc_parser.read_file("ISC-RB", "ISC-Global")
print("ISC Reviewed Bulleting Catalogue contains: %d events" % isc_catalogue.get_number_events())

iscgem_parser = GenericCataloguetoISFParser("inputs/isc-gem-cat-3.csv")
iscgem_catalogue = iscgem_parser.parse("ISC-GEM", "ISC-GEM-CAT")

print("ISC-GEM Catalogue contains: %d events" % iscgem_catalogue.get_number_events())

ISC Reviewed Bulleting Catalogue contains: 68488 events
ISC-GEM Catalogue contains: 3993 events


In [ ]:
isc_catalogue

In [24]:
merger = DuplicateFinder(iscgem_catalogue, 0.0001, 0.0001, logging=True)
mergedCatalogue = merger.merge_catalogue(isc_catalogue)

After duplicate finding: 72478 events (72478)


In [ ]:
## Define Rule Sets

The catalogue covers the years 2005/06. To illustrate how to apply time variable hierarchies we consider two set of rules:

For the origin the order of preference is:

(For 2005): EHB, ISC, NEIC, GCMT/HRVD, BJI

(For 2006): ISC, EHB, NEIC, BJI, GCMT/HRVD


In [25]:
# origin_rules = [
#     ("2015/01/01 - 2021/12/31", ["EHB", "ISC", "NEIC", "GCMT"]),
#     ("2021/01/01 - 2007/01/01", ["EHB", "ISC", "NEIC", "GCMT"])
# ]

origin_rules = [
    ("1900/01/01 - 2021/12/31", ["ISC-GEM", "ISC-EHB", "EHB", "ISC", "IDC", "NEIC", "NEIS", "USCGS", "NIED", "GCMT", "GUTE", "PAS"])
]

### Magnitude Rules 

#### GCMT/HRVD

In [ ]:
def gcmt_hrvd_mw(magnitude):
    """
    For Mw recorded by GCMT take the value with no uncertainty
    """
    return magnitude

def gcmt_hrvd_mw_sigma(magnitude):
    """
    No additional uncertainty   
    """
    return 0.0

#### ISC/NEIC

#### Weatherill (2015) Table 1. Mw conversions

In [26]:
"""
Weatherill (2015) Table 1. Mw conversions
"""
def iscgem_mw(magnitude):
    """
    For Mw recorded by ISCGEM take the value with no uncertainty
    """
    return magnitude

def iscgem_mw_sigma(magnitude):
    """
    No additional uncertainty   
    """
    return 0.0

def gcmt_mw(magnitude):
    """
    For Mw recorded by GCMT take the value with no uncertainty
    """
    return magnitude

def gcmt_mw_sigma(magnitude):
    """
    No additional uncertainty   
    """
    return 0.0

def neic_mw(magnitude):
    """
    If Mw reported by NEIC,
    """
    return 1.021 * magnitude - 0.091

def neic_mw_sigma(magnitude):
    """
    Uncertainty of 0.101 units
    """
    return 0.105

def nied_mw(magnitude):
    """
    If Mw reported by NIED,
    """
    return 0.964 * magnitude + 0.248

def nied_mw_sigma(magnitude):
    """
    Uncertainty of 0.11 units
    """
    return 0.11

def isc_ms(magnitude):
    """
    If Ms reported by ISC, convert to Mw from Weatherill (2015),
    """
    if magnitude > 6.0:
        return 0.994 * magnitude + 0.1        
    else:
        return 0.616 * magnitude + 2.369

def isc_ms_sigma(magnitude):
    """
    With Magnitude dependent uncertainty
    """
    if magnitude > 6.0:
        return 0.174
    else:
        return 0.147

def neic_ms(magnitude):
    """
    If Ms reported by NEIC, convert to Mw from Weatherill (2015),
    """
    if magnitude > 6.47:
        return 1.005 * magnitude - 0.026       
    else:
        return 0.723 * magnitude + 1.798

def neic_ms_sigma(magnitude):
    """
    With Magnitude dependent uncertainty
    """
    if magnitude > 6.47:
        return 0.187
    else:
        return 0.159
    
def neic_msz(magnitude):
    """
    If Msz reported by NEIC, convert to Mw from Weatherill (2015),
    """
    if magnitude > 6.47:
        return 0.950 * magnitude + 0.359     
    else:
        return 0.707 * magnitude + 1.933

def neic_msz_sigma(magnitude):
    """
    With Magnitude dependent uncertainty
    """
    if magnitude > 6.47:
        return 0.204
    else:
        return 0.179

def neic_mb(magnitude):
    """
    If Mb reported by NEIC,
    """
    return 1.159 * magnitude - 0.659

def neic_mb_sigma(magnitude):
    """
    Uncertainty of 0.283 units
    """
    return 0.283

def isc_mb(magnitude):
    """
    If Mw reported by isc,
    """
    return 1.084 * magnitude - 0.142

def isc_mb_sigma(magnitude):
    """
    Uncertainty of 0.317 units
    """
    return 0.317

def pas_ms(magnitude):
    """
    For Ms recorded by paS take the value with no uncertainty. 
    In their database Pacheco & Sykes (1992) use
    the 20-s period Ms value, which, for our purposes, we treat as
    equivalent to MW in the magnitude range 7.0 ≤ MW ≤ 8.0. (Weatherill, 2015)
    """
    return magnitude

def pas_ms_sigma(magnitude):
    """
    0.2 additional uncertainty   
    """
    return 0.2


### Define Magnitude Hierarchy

In [27]:
# rule_set_2015 = [
#     MagnitudeConversionRule("GCMT", "Mw", gcmt_hrvd_mw, gcmt_hrvd_mw_sigma),
#     MagnitudeConversionRule("HRVD", "Mw", gcmt_hrvd_mw, gcmt_hrvd_mw_sigma),
#     MagnitudeConversionRule("ISC", "Ms", scordillis_ms, scordillis_ms_sigma),
#     MagnitudeConversionRule("NEIC", "Ms", scordillis_ms, scordillis_ms_sigma),
#     MagnitudeConversionRule("ISC", "mb", scordillis_mb, scordillis_mb_sigma),
#     MagnitudeConversionRule("NEIC", "mb", scordillis_mb, scordillis_mb_sigma),
#     MagnitudeConversionRule("BJI", "Ms", bji_ms, bji_ms_sigma),
#     MagnitudeConversionRule("BJI", "mb", bji_mb, bji_mb_sigma)
# ]

# rule_set_2006 = [
#     MagnitudeConversionRule("GCMT", "Mw", gcmt_hrvd_mw, gcmt_hrvd_mw_sigma),
#     MagnitudeConversionRule("HRVD", "Mw", gcmt_hrvd_mw, gcmt_hrvd_mw_sigma),
#     MagnitudeConversionRule("ISC", "Ms", scordillis_ms, scordillis_ms_sigma),
#     MagnitudeConversionRule("BJI", "Ms", bji_ms, bji_ms_sigma),
#     MagnitudeConversionRule("NEIC", "Ms", scordillis_ms, scordillis_ms_sigma),
#     MagnitudeConversionRule("ISC", "mb", scordillis_mb, scordillis_mb_sigma),
#     MagnitudeConversionRule("BJI", "mb", bji_mb, bji_mb_sigma),
#     MagnitudeConversionRule("NEIC", "mb", scordillis_mb, scordillis_mb_sigma)
#]

rule_set_1900 = [
    MagnitudeConversionRule("ISC-GEM", "Mw", iscgem_mw, iscgem_mw_sigma),
    MagnitudeConversionRule("GCMT", "Mw", gcmt_mw, gcmt_mw_sigma),
    MagnitudeConversionRule("NEIC", "Mw", neic_mw, neic_mw_sigma),
    MagnitudeConversionRule("NIED", "Mw", nied_mw, nied_mw_sigma),
    MagnitudeConversionRule("ISC", "Ms", isc_ms, isc_ms_sigma),
    MagnitudeConversionRule("IDC", "Ms", isc_ms, isc_ms_sigma),
    MagnitudeConversionRule("NEIC", "Ms", neic_ms, neic_ms_sigma),
    MagnitudeConversionRule("USCGS", "Ms", neic_ms, neic_ms_sigma),
    MagnitudeConversionRule("NEIC", "Msz", neic_msz, neic_msz_sigma),
    MagnitudeConversionRule("NEIC", "Mb", neic_mb, neic_mb_sigma),
    MagnitudeConversionRule("USCGS", "Mb", neic_mb, neic_mb_sigma),
    MagnitudeConversionRule("ISC", "Mb", isc_mb, isc_mb_sigma),
    MagnitudeConversionRule("PAS", "Ms", pas_ms, pas_ms_sigma),
]

# magnitude_rules = [
#     ("2015/01/01 - 2021/12/31", rule_set_2005),
#     ("2006/01/01 - 2007/01/01", rule_set_2006)
# ]

magnitude_rules = [
    ("1900/01/01 - 2021/12/31", rule_set_1900)
]

### Pre-processing

Before executing the homogenisation it is necessary to run a preprocessing step. This searches through the catalogue and identifies which conversion rule to apply:

The preprocessor is instantiated with a string describing the sort of rules to be applied.

"time" - Applies time only

"key" - Applies key rules only

"depth" - Applies depth rules only

"time|key" - Applies joint time and key rules

"time|depth" - Applied joint time and depth rules

"depth|key" - Applies joint depth and key rules

In [28]:
preprocessor = HomogenisorPreprocessor("time")
catalogue = preprocessor.execute(mergedCatalogue, origin_rules, magnitude_rules)
# catalogue = preprocessor.execute(isc_catalogue, origin_rules, magnitude_rules)

### Harmonise the Catalogue

In [29]:
harmonisor = DynamicHomogenisor(catalogue, logging=True)
homogenised_catalogue = harmonisor.homogenise(magnitude_rules, origin_rules)

610548604 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610548608 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610548621 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957889 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957892 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957904 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957903 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957908 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610548662 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610548664 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957926 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
914258 PAS-Ms GUTE|PAS-Ms
914262 PAS-Ms GUTE|PAS-Ms
16957953 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
914263 PAS-Ms GUTE|PAS-Ms
16957952 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610548683 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610548685 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610548686 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957964 PAS-Ms ISC|PAS-Ms
16957967 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957968 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610548699 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610548707 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
610326336 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957982 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
16957985 ISC-GEM-Mw ISC-GE

607997013 IDC-Ms ISC-EHB|IDC-Ms
608100943 IDC-Ms ISC|IDC-Ms
608100959 ISC-Ms ISC|ISC-Ms
611838225 GCMT-Mw ISC-EHB|GCMT-Mw
608101011 ISC-Mb ISC|ISC-Mb
608101017 -- None --  None
608101019 IDC-Ms ISC|IDC-Ms
608101024 IDC-Ms ISC|IDC-Ms
608101030 IDC-Ms ISC|IDC-Ms
608101032 -- None --  None
608101057 ISC-Mb ISC|ISC-Mb
608101061 IDC-Ms ISC|IDC-Ms
608101062 IDC-Ms IDC|IDC-Ms
608101083 IDC-Ms ISC|IDC-Ms
608101092 IDC-Ms ISC|IDC-Ms
608128306 ISC-Mb ISC|ISC-Mb
608005035 IDC-Ms ISC-EHB|IDC-Ms
612296663 IDC-Ms ISC-EHB|IDC-Ms
608128310 IDC-Ms ISC|IDC-Ms
608128324 IDC-Ms ISC|IDC-Ms
608128329 -- None --  None
608005673 GCMT-Mw ISC-EHB|GCMT-Mw
608005673 ISC-GEM-Mw ISC-GEM|ISC-GEM-Mw
608128342 IDC-Ms ISC|IDC-Ms
608128346 -- None --  None
608128347 -- None --  None
608128349 IDC-Ms ISC|IDC-Ms
608128353 NEIC-Mb ISC|NEIC-Mb
608128367 ISC-Mb ISC|ISC-Mb
608128369 IDC-Ms IDC|IDC-Ms
608128372 IDC-Ms ISC|IDC-Ms
608128373 -- None --  None
608128377 IDC-Ms ISC-EHB|IDC-Ms
608128385 IDC-Ms ISC|IDC-Ms
608128388 IS

617436213 IDC-Ms IDC|IDC-Ms
617436218 IDC-Ms ISC|IDC-Ms
617436236 IDC-Ms ISC-EHB|IDC-Ms
617436246 IDC-Ms ISC|IDC-Ms
617436252 ISC-Mb ISC|ISC-Mb
617436278 ISC-Mb ISC|ISC-Mb
617436341 ISC-Mb ISC|ISC-Mb
616918607 ISC-Ms ISC-EHB|ISC-Ms
617436353 IDC-Ms ISC|IDC-Ms
617436360 IDC-Ms ISC-EHB|IDC-Ms
617436361 NEIC-Mb ISC|NEIC-Mb
617436383 IDC-Ms ISC|IDC-Ms
617436414 ISC-Mb ISC|ISC-Mb
617436428 -- None --  None
617436488 NEIC-Mb ISC|NEIC-Mb
617436511 IDC-Ms ISC|IDC-Ms
617436513 ISC-Mb ISC|ISC-Mb
617436602 IDC-Ms IDC|IDC-Ms
617436607 ISC-Mb ISC|ISC-Mb
617436615 -- None --  None
617436625 ISC-Mb ISC|ISC-Mb
617436664 IDC-Ms ISC|IDC-Ms
617436680 IDC-Ms ISC|IDC-Ms
617436683 IDC-Ms ISC|IDC-Ms
617436702 -- None --  None
616970677 NIED-Mw ISC|NIED-Mw
617436724 ISC-Mb ISC|ISC-Mb
617436733 IDC-Ms ISC|IDC-Ms
617436736 ISC-Mb ISC|ISC-Mb
617436745 ISC-Ms ISC|ISC-Ms
617436748 IDC-Ms IDC|IDC-Ms
617436774 -- None --  None
617436777 -- None --  None
617436780 ISC-Mb ISC|ISC-Mb
617436786 ISC-Ms ISC|ISC-Ms
6174367

As logging was enabled, we can dump the log to a csv file and explore which rules and which hierarchy was applied for each event

In [30]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now().strftime('%Y.%m.%d.%H.%M.%S')

log_file = f"outputs/1900-2021-isc-gem-rb-homogenisor_log_{current_datetime}.csv"
# log_file = f"outputs/1900-2021-isc-rb-homogenisor_log_{current_datetime}.csv"
if os.path.exists(log_file):
    os.remove(log_file)

harmonisor.dump_log(log_file)

TypeError: argument of type 'NoneType' is not iterable

### Export the Homogenised Catalogue to CSV

In [1]:
output_catalogue_file = f"outputs/1900-2021-isc-gem-rb-homogeneous_catalogue_{current_datetime}.csv"
# output_catalogue_file = f"outputs/1900-2021-isc-rb-homogeneous_catalogue_{current_datetime}.csv"
if os.path.exists(output_catalogue_file):
    os.remove(output_catalogue_file)
harmonisor.export_homogenised_to_csv(output_catalogue_file)

NameError: name 'current_datetime' is not defined

In [11]:
import os
import pandas as pd

In [8]:
csv_file_path = os.getcwd() + r'\inputs\datetime-test.csv'
csv_file_path

'C:\\Users\\enrico.abcede\\mbtk\\oq-mbtk\\notebook\\datetime-test.csv'

In [23]:
# Replace 'your_file.csv' with the actual path to your CSV file.
csv_file_path = os.getcwd() + r'\inputs\datetime-weatherill.csv'

# Use read_csv() to read the CSV file and create a DataFrame.
df = pd.read_csv(csv_file_path)


# Create a pandas datetime object by combining the individual columns.
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute', 'second']])

# Get the Unix timestamp and store it in a new column 'unix_timestamp'.
df['unix_weatherill-timestamp'] = df['datetime'].apply(lambda x: x.timestamp())

# Now you have a new column 'unix_timestamp' containing the Unix timestamps.
df

# Define the filename for the CSV file.
csv_file_outpath = os.getcwd() + r'\outputs'
csv_filename = "weatherill-unix.csv"

# Concatenate the current directory and the filename to get the full file path.
output_path = os.path.join(csv_file_outpath , csv_filename)

# Save the DataFrame as a CSV file in the specified output path.
df.to_csv(output_path, index=False)